In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [2]:
fips_check_scraped = pd.read_csv("../../../data/state_data/geo/geo_fips/23/CA_fips_scraped.csv") 
fips_check_scraped  = fips_check_scraped.loc[:, ~fips_check_scraped.columns.str.contains('^Unnamed')] 
fips_check_scraped.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,6223567700,CA,"1661 MARTIN RANCH RD, SAN BERNARDINO, CA",34.224719,-117.372550,6.071005e+13
1,1048448901,CA,"6120 Stoneridge Mall Rd Ste 300, Pleasanton, CA",37.698909,-121.924918,6.001451e+13
2,1122827105,CA,"431 ISIS AVE, INGLEWOOD, CA",33.957520,-118.375870,6.037601e+13
3,1187837109,CA,"31752 COAST HWY, STE 300, LAGUNA BEACH, CA",33.501729,-117.743448,6.059042e+13
4,1321898800,CA,"2171 Junipero Serra Blvd, Daly City, CA",37.698365,-122.470386,6.081601e+13


In [3]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,6223567700,CA,"1661 MARTIN RANCH RD, SAN BERNARDINO, CA",34.224719,-117.372550,6.071005e+13
1,1048448901,CA,"6120 Stoneridge Mall Rd Ste 300, Pleasanton, CA",37.698909,-121.924918,6.001451e+13
2,1122827105,CA,"431 ISIS AVE, INGLEWOOD, CA",33.957520,-118.375870,6.037601e+13
3,1187837109,CA,"31752 COAST HWY, STE 300, LAGUNA BEACH, CA",33.501729,-117.743448,6.059042e+13
4,1321898800,CA,"2171 Junipero Serra Blvd, Daly City, CA",37.698365,-122.470386,6.081601e+13


In [4]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #28093

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
1596,3096957408,CA,"1669 flanigan dr 108, SAN JOSE, CA",37.318528,-121.824322,NaN
1734,3413037103,CA,"1041 Formosa Avenue, WEST HOLLYWOOD, CA",34.090229,-118.346474,NaN
1826,3512057202,CA,"5110 Telegraph Avenue, OAKLAND, CA",37.837412,-122.262167,NaN
2851,8850497200,CA,"18900 Studebaker Road, Cerritos, CA",33.859873,-118.098688,NaN
3081,7723857008,CA,"4360 Park Terrace Dr Ste 140, WESTLAKE VILLAGE...",34.144678,-118.800550,NaN
...,...,...,...,...,...,...
124725,5710847704,CA,"4160 HACIENDA DR STE 200, PLEASANTON, CA",37.686565,-121.895304,NaN
125468,7407998510,CA,"INTEGRATED INTERIORS INC. 04/01/2017, ANAHEIM, CA",33.836593,-117.914301,NaN
126187,5287997705,CA,"7657 WINNETKA AVE #444, WINNETKA, CA",34.210305,-118.571491,NaN
128258,3634637110,CA,"712 WALKER STREET, WATSONVILLE, CA",36.910904,-121.765737,NaN


<h4>FIPS Script

In [5]:
#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [6]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('06')] #CA
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

  0%|          | 0/114 [00:00<?, ?it/s]

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [7]:
geo_id_list =geo_id 

Length Check

In [8]:
print(len(geo_id_list))

114


Matching FIPS to DataFrame

In [10]:
fips_fill = fips_check_scraped_null

In [11]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_5546/597263667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
1596,3096957408,CA,"1669 flanigan dr 108, SAN JOSE, CA",37.318528,-121.824322,[060855033041004]
1734,3413037103,CA,"1041 Formosa Avenue, WEST HOLLYWOOD, CA",34.090229,-118.346474,[060377001022002]
1826,3512057202,CA,"5110 Telegraph Avenue, OAKLAND, CA",37.837412,-122.262167,[060014003002021]
2851,8850497200,CA,"18900 Studebaker Road, Cerritos, CA",33.859873,-118.098688,[060375545212004]
3081,7723857008,CA,"4360 Park Terrace Dr Ste 140, WESTLAKE VILLAGE...",34.144678,-118.800550,[060378003351008]
...,...,...,...,...,...,...
124725,5710847704,CA,"4160 HACIENDA DR STE 200, PLEASANTON, CA",37.686565,-121.895304,[060014507432046]
125468,7407998510,CA,"INTEGRATED INTERIORS INC. 04/01/2017, ANAHEIM, CA",33.836593,-117.914301,[060590873024002]
126187,5287997705,CA,"7657 WINNETKA AVE #444, WINNETKA, CA",34.210305,-118.571491,[060371341042005]
128258,3634637110,CA,"712 WALKER STREET, WATSONVILLE, CA",36.910904,-121.765737,[060871104011002]


Remove brackets

In [12]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_5546/1219885402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
1596,3096957408,CA,"1669 flanigan dr 108, SAN JOSE, CA",37.318528,-121.824322,060855033041004
1734,3413037103,CA,"1041 Formosa Avenue, WEST HOLLYWOOD, CA",34.090229,-118.346474,060377001022002
1826,3512057202,CA,"5110 Telegraph Avenue, OAKLAND, CA",37.837412,-122.262167,060014003002021
2851,8850497200,CA,"18900 Studebaker Road, Cerritos, CA",33.859873,-118.098688,060375545212004
3081,7723857008,CA,"4360 Park Terrace Dr Ste 140, WESTLAKE VILLAGE...",34.144678,-118.800550,060378003351008
...,...,...,...,...,...,...
124725,5710847704,CA,"4160 HACIENDA DR STE 200, PLEASANTON, CA",37.686565,-121.895304,060014507432046
125468,7407998510,CA,"INTEGRATED INTERIORS INC. 04/01/2017, ANAHEIM, CA",33.836593,-117.914301,060590873024002
126187,5287997705,CA,"7657 WINNETKA AVE #444, WINNETKA, CA",34.210305,-118.571491,060371341042005
128258,3634637110,CA,"712 WALKER STREET, WATSONVILLE, CA",36.910904,-121.765737,060871104011002


In [13]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
16010,4861237004,CA,"2865 South Jones Blvd, SHERMAN OAKS, CA",36.137302,-115.225643,NaN
46049,5428878008,CA,"122 ORCHARD DR, CAMPBELL, CA",40.080060,-80.889337,NaN
56097,6471047010,CA,"47110 WASHINGTON ST STE 201, LA QUINTA, CA",46.572476,-122.905019,NaN
57592,7948857804,CA,"19912 Stateline Road, Tulelake, CA",41.997363,-121.435929,NaN
60769,6089678409,CA,"19912 Stateline Rd, Tulelake, CA",41.997363,-121.435929,NaN
76035,1093197302,CA,"5959 Commerce Blvd, Isle-Aux-Morts, CA",47.587461,-58.974439,NaN
88686,7414237300,CA,"1310 Millet St, Anaheim, CA",36.634356,-87.301086,NaN
98484,5027317007,CA,"2865 South Jones, Blvd., SHERMAN OAKS, CA",36.137302,-115.225643,NaN
112283,9361287200,CA,"8816 FOOTHILL BLVD #103-131, RANCHO CUCAMONGA, CA",29.875339,-152.279104,NaN
113073,4126958607,CA,"8816 Foothill Blvd Ste 103-393, Rancho Cucamon...",29.875339,-152.279104,NaN


Parse for Merge

In [14]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
1596,3096957408,060855033041004
1734,3413037103,060377001022002
1826,3512057202,060014003002021
2851,8850497200,060375545212004
3081,7723857008,060378003351008
...,...,...
124725,5710847704,060014507432046
125468,7407998510,060590873024002
126187,5287997705,060371341042005
128258,3634637110,060871104011002


Read in base data

In [15]:

#if from FIPS file
fips_check = pd.read_csv("../../../data/state_data/geo/geo_fips/23/CA_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
fips_check.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,35208,6223567700,CA,"1661 MARTIN RANCH RD, SAN BERNARDINO, CA",34.224719,-117.372550,6.071005e+13
1,35209,1048448901,CA,"6120 Stoneridge Mall Rd Ste 300, Pleasanton, CA",37.698909,-121.924918,6.001451e+13
2,35210,1122827105,CA,"431 ISIS AVE, INGLEWOOD, CA",33.957520,-118.375870,6.037601e+13
3,35211,1187837109,CA,"31752 COAST HWY, STE 300, LAGUNA BEACH, CA",33.501729,-117.743448,6.059042e+13
4,35212,1321898800,CA,"2171 Junipero Serra Blvd, Daly City, CA",37.698365,-122.470386,6.081601e+13


Merge with MainFrame

In [32]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,35208,6223567700,CA,"1661 MARTIN RANCH RD, SAN BERNARDINO, CA",34.224719,-117.372550,6.071005e+13,NaN
1,35209,1048448901,CA,"6120 Stoneridge Mall Rd Ste 300, Pleasanton, CA",37.698909,-121.924918,6.001451e+13,NaN
2,35210,1122827105,CA,"431 ISIS AVE, INGLEWOOD, CA",33.957520,-118.375870,6.037601e+13,NaN
3,35211,1187837109,CA,"31752 COAST HWY, STE 300, LAGUNA BEACH, CA",33.501729,-117.743448,6.059042e+13,NaN
4,35212,1321898800,CA,"2171 Junipero Serra Blvd, Daly City, CA",37.698365,-122.470386,6.081601e+13,NaN
...,...,...,...,...,...,...,...,...
129081,164289,9980578504,CA,"16633 Ventura Blvd Ste 913, Encino, CA",34.158291,-118.495278,6.037140e+13,NaN
129082,164290,9985047708,CA,"10749 ROSELLE ST, SAN DIEGO, CA",32.899195,-117.222392,6.073008e+13,NaN
129083,164291,9985407709,CA,"11180 WARNER AVENUE, FOUNTAIN VALLEY, CA",33.715426,-117.933275,6.059099e+13,NaN
129084,164292,9989087303,CA,"946 Santa Rosa Ave, SANTA ROSA, CA",38.429187,-122.711481,6.097151e+13,NaN


<b>Fill FIPS Columns - if FIPS file

In [33]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,35208,6223567700,CA,"1661 MARTIN RANCH RD, SAN BERNARDINO, CA",34.224719,-117.372550,6.071005e+13,NaN,60710045031000.0
1,35209,1048448901,CA,"6120 Stoneridge Mall Rd Ste 300, Pleasanton, CA",37.698909,-121.924918,6.001451e+13,NaN,60014506091002.0
2,35210,1122827105,CA,"431 ISIS AVE, INGLEWOOD, CA",33.957520,-118.375870,6.037601e+13,NaN,60376014013017.0
3,35211,1187837109,CA,"31752 COAST HWY, STE 300, LAGUNA BEACH, CA",33.501729,-117.743448,6.059042e+13,NaN,60590423051032.0
4,35212,1321898800,CA,"2171 Junipero Serra Blvd, Daly City, CA",37.698365,-122.470386,6.081601e+13,NaN,60816006001007.0


Drop Extra FIPS Columns

In [34]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0"], axis=1)
fips_merge_drop

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,35208,6223567700,CA,"1661 MARTIN RANCH RD, SAN BERNARDINO, CA",34.224719,-117.372550,60710045031000.0
1,35209,1048448901,CA,"6120 Stoneridge Mall Rd Ste 300, Pleasanton, CA",37.698909,-121.924918,60014506091002.0
2,35210,1122827105,CA,"431 ISIS AVE, INGLEWOOD, CA",33.957520,-118.375870,60376014013017.0
3,35211,1187837109,CA,"31752 COAST HWY, STE 300, LAGUNA BEACH, CA",33.501729,-117.743448,60590423051032.0
4,35212,1321898800,CA,"2171 Junipero Serra Blvd, Daly City, CA",37.698365,-122.470386,60816006001007.0
...,...,...,...,...,...,...,...
129081,164289,9980578504,CA,"16633 Ventura Blvd Ste 913, Encino, CA",34.158291,-118.495278,60371397012008.0
129082,164290,9985047708,CA,"10749 ROSELLE ST, SAN DIEGO, CA",32.899195,-117.222392,60730083391037.0
129083,164291,9985407709,CA,"11180 WARNER AVENUE, FOUNTAIN VALLEY, CA",33.715426,-117.933275,60590992294002.0
129084,164292,9989087303,CA,"946 Santa Rosa Ave, SANTA ROSA, CA",38.429187,-122.711481,60971514061015.0


In [35]:
fips_merge_drop.loc[fips_merge_drop["FIPS_z"].isnull()] #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
16010,4861237004,CA,"2865 South Jones Blvd, SHERMAN OAKS, CA",36.137302,-115.225643,NaN
46049,5428878008,CA,"122 ORCHARD DR, CAMPBELL, CA",40.080060,-80.889337,NaN
56097,6471047010,CA,"47110 WASHINGTON ST STE 201, LA QUINTA, CA",46.572476,-122.905019,NaN
57592,7948857804,CA,"19912 Stateline Road, Tulelake, CA",41.997363,-121.435929,NaN
60769,6089678409,CA,"19912 Stateline Rd, Tulelake, CA",41.997363,-121.435929,NaN
76035,1093197302,CA,"5959 Commerce Blvd, Isle-Aux-Morts, CA",47.587461,-58.974439,NaN
88686,7414237300,CA,"1310 Millet St, Anaheim, CA",36.634356,-87.301086,NaN
98484,5027317007,CA,"2865 South Jones, Blvd., SHERMAN OAKS, CA",36.137302,-115.225643,NaN
112283,9361287200,CA,"8816 FOOTHILL BLVD #103-131, RANCHO CUCAMONGA, CA",29.875339,-152.279104,NaN
113073,4126958607,CA,"8816 Foothill Blvd Ste 103-393, Rancho Cucamon...",29.875339,-152.279104,NaN


Convert to File

In [36]:
fips_CA = fips_merge_drop

In [38]:
fips_CA.to_csv("../../../data/state_data/geo/geo_fips/23/CA_fips_scraped.csv") 